# [Interpret](https://github.com/interpretml/interpret)

InterpretML is an open-source package that incorporates state-of-the-art machine learning interpretability techniques under one roof.    
With this package, you can train interpretable glassbox models and explain blackbox systems.    
InterpretML helps you understand your model's global behavior, or understand the reasons behind individual predictions.



In [1]:
import warnings

import vaex
from vaex.ml.datasets import load_iris_1e5

warnings.filterwarnings('ignore')

df = load_iris_1e5()
target = 'class_'

# feature engineering example
df['petal_ratio'] = df['petal_length'] / df['petal_width']

df[target] = df[target].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

## Modeling

In [2]:
from interpret.glassbox import ExplainableBoostingClassifier

features = ['petal_length', 'petal_width', 'sepal_length', 'sepal_width', 'petal_ratio']

ebm = ExplainableBoostingClassifier(feature_names=features)

ebm.fit(df[features].values, df[target].tolist())


ExplainableBoostingClassifier(feature_names=['petal_length', 'petal_width',
                                             'sepal_length', 'sepal_width',
                                             'petal_ratio'],
                              feature_types=['continuous', 'continuous',
                                             'continuous', 'continuous',
                                             'continuous'],
                              interactions=0)

In [3]:



# add predictions to dataframe
@vaex.register_function(on_expression=False)
def predict(*columns):
    return ebm.predict(np.array(columns).T)


df.add_function("predict", predict)
df["predictions"] = df.func.predict(*features)
df.head(2)

#,sepal_length,sepal_width,petal_length,petal_width,class_,petal_ratio,predictions
0,5.9,3,4.2,1.5,versicolor,2.8,versicolor
1,6.1,3,4.6,1.4,versicolor,3.28571,versicolor


In [ ]:
from collections import defaultdict
import pyarrow as pa


@vaex.register_function(on_expression=False)
def explain(*columns):
    data = np.array(columns).T
    explanation = ebm.explain_local(data)._internal_obj['specific']
    label_names = explanation['meta']['label_names']
    for e in explanation:
        ret = []
        tmp = defaultdict(dict)
        for feature, score in zip(e['names'], e['scores']):
            for i, label in enumerate(label_names):
                tmp[feature][label] = score[i]
            ret.append(dict(tmp))
    return pa.array(ret)


df.add_function("explain", explain)
df["explain"] = df.func.explain(*features)
df.head(2)

In [ ]:
# Vaex remember all the transformations, this is a skleran.pipeline alternative
pipeline = Pipeline.from_vaex(df, description='simple lightGBM')
pipeline.raw.pop(target)  # (optional) we don't expect to get the class_ in queries
assert pipeline.validate()
print("Pipeline raw data example:")
print(json.dumps(pipeline.raw, indent=4))
print('')
print("Pipeline output example:")
pipeline.inference(pipeline.raw).to_records()
df.head(2)

In [ ]:
import vaex
import warnings
from vaex.ml.datasets import load_iris_1e5
from vaex.ml.lightgbm import LightGBMModel
from goldilox import Pipeline
import numpy as np
import json

warnings.filterwarnings('ignore')

df = load_iris_1e5()
target = 'class_'

# feature engineering example
df['petal_ratio'] = df['petal_length'] / df['petal_width']

booster = LightGBMModel(features=['petal_length', 'petal_width', 'sepal_length', 'sepal_width', 'petal_ratio'],
                        target=target,
                        prediction_name='predictions',
                        num_boost_round=500, params={'verbosity': -1,
                                                     'objective': 'multiclass',
                                                     'num_class': 3})
booster.fit(df)
df = booster.transform(df)


# post model processing example
@vaex.register_function()
def argmax(ar, axis=1):
    return np.argmax(ar, axis=axis)


df.add_function('argmax', argmax)
df['prediction'] = df['predictions'].argmax()

df['label'] = df['prediction'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

# Vaex remember all the transformations, this is a skleran.pipeline alternative
pipeline = Pipeline.from_vaex(df, description='simple lightGBM')
pipeline.raw.pop(target)  # (optional) we don't expect to get the class_ in queries
assert pipeline.validate()
print("Pipeline raw data example:")
print(json.dumps(pipeline.raw, indent=4))
print('')
print("Pipeline output example:")
pipeline.inference(pipeline.raw).to_records()
df.head(2)

In [ ]:
pipeline.raw.pop('class_', None)
print(f"Saved to: {pipeline.save('../tests/models/server.pkl')}")
print(f"Check out the docs: http://127.0.0.1:5000\n")

!gl serve../ tests / models / server.pkl